In [1]:
import rospy
import rospkg
import os

from nav_msgs.srv import GetMap

In [2]:
rospy.set_param("/MARL", True)
rospy.init_node(f"USER_NODE", anonymous=True)

# RobotManager Tests

In [3]:
service_client_get_map = rospy.ServiceProxy('/static_map', GetMap)
map_response = service_client_get_map()
models_folder_path = rospkg.RosPack().get_path('simulator_setup')
robot_yaml = os.path.join(models_folder_path, 'robot', "myrobot.model.yaml")

In [4]:
from task_generator.robot_manager import RobotManager
from rl_agent.utils.observation_collector import ObservationCollector

# namespace convention: *simulation ns*/*robot name*/*data topic*
# robot1 namespace: '/sim_1/test1/...' e.g. '/sim_1/test1/goal', '/sim_1/test1/odom' etc.
robot1 = RobotManager(ns='sim_1', map_=map_response.map, robot_yaml_path=robot_yaml, robot_name='test1')
robot2 = RobotManager(ns='sim_1', map_=map_response.map, robot_yaml_path=robot_yaml, robot_name='test2')

In [5]:
# sets random start and goal position and returns the poses
start_pos_1, goal_pos_1 = robot1.set_start_pos_goal_pos()
start_pos_2, goal_pos_2 = robot2.set_start_pos_goal_pos()
print(f"{goal_pos_1}\n{goal_pos_2:}")

x: 0.4500000961124897
y: -2.0499999411404133
theta: -1.8848128590685944
x: 4.100000150501728
y: -1.3499999307096004
theta: -1.1108516303702127


# TASK MANAGER TESTS

In [ ]:
from task_generator.marl_tasks import get_MARL_task

# number of static and dynamic obstacles still hardcoded for this task mode
task_manager = get_MARL_task(
    "sim_1", "random", robot_names=["test1", "test2"]
)

[INFO] [1629843060.130664, 0.000000]: Registed 8 dynamic obstacles and 12 static obstacles


In [ ]:
# generates random dynamic and static obstacle configurations as well as
# random start and goal pose
task_manager.reset()

# DRL AGENT CLASS TESTS

In [3]:
DEFAULT_HYPERPARAMETER = os.path.join(
    rospkg.RosPack().get_path("arena_local_planner_drl"),
    "configs",
    "hyperparameters",
    "default.json",
)

In [4]:
from rl_agent.training_agent_wrapper import TrainingDRLAgent

agent1 = TrainingDRLAgent(
    ns="sim_1", 
    robot_name="test1",
    hyperparameter_path=DEFAULT_HYPERPARAMETER,
)
agent2 =TrainingDRLAgent(
    ns="sim_1", 
    robot_name="test2",
    hyperparameter_path=DEFAULT_HYPERPARAMETER,
)

/home/tuananhroman/python_env/rosnav/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [14]:
# in order to retrieve the first observation, manually call the StepWorld service
# as follows: rosservice call /sim_1/step_world "0.1"  
agent1.get_observations()

(array([ 3.45155525,  3.41534781,  3.43184376,  3.43339014,  3.4159615 ,
         3.43589115,  3.4039278 ,  3.42613411,  3.39578891,  3.36605525,
         3.39161563,  3.36347795,  3.39151049,  3.42106676,  3.4522028 ,
         3.5       ,  3.5       ,  3.5       ,  3.5       ,  3.5       ,
         3.5       ,  3.5       ,  3.5       ,  3.5       ,  3.5       ,
         3.5       ,  3.5       ,  3.5       ,  3.5       ,  3.5       ,
         3.5       ,  3.5       ,  3.5       ,  3.5       ,  3.5       ,
         3.5       ,  3.5       ,  3.5       ,  1.77824557,  1.75453162,
         1.73196244,  1.71048105,  1.69003451,  1.67057371,  1.65205359,
         1.63443208,  1.61767006,  1.6017313 ,  1.58658218,  1.57219124,
         1.5585295 ,  1.5455699 ,  1.53328717,  1.5216583 ,  1.51066089,
         1.50027514,  1.49048209,  1.48126435,  1.47260571,  1.46449113,
         1.45690703,  1.44984043,  1.4432795 ,  1.43721342,  1.4316324 ,
         1.4265275 ,  1.4218905 ,  1.41771388,  1.4

In [5]:
agent_list = [agent1, agent2]

from rl_agent.envs.pettingzoo_env import FlatlandPettingZooEnv

env = FlatlandPettingZooEnv(ns="sim_1", agent_list=agent_list)

[INFO] [1630358333.688796, 0.000000]: Registed 10 dynamic obstacles and 0 static obstacles


In [6]:
import numpy as np

In [19]:
env.reset()

{'test1': array([ 2.07239079,  1.9686743 ,  1.87538815,  1.7910645 ,  1.71449697,
         1.64468825,  1.58080471,  1.52214396,  1.46811247,  1.418203  ,
         1.37197936,  1.32906592,  1.28913593,  1.25190568,  1.21712589,
         1.18457735,  1.15406668,  1.12542272,  1.09849238,  1.07314026,
         1.04924405,  1.02669501,  1.0053947 ,  0.98525411,  0.96619314,
         0.94813895,  0.93102568,  0.91479284,  0.89938569,  0.88475376,
         0.87085146,  0.85763651,  0.84507006,  0.83311677,  0.82174385,
         0.81092083,  0.80061984,  0.79081529,  0.78148299,  0.77260107,
         0.76414883,  0.75610757,  0.7484591 ,  0.74118745,  0.73427731,
         0.72771448,  0.72148573,  0.71557868,  0.70998228,  0.70468569,
         0.69967902,  0.69495314,  0.6904996 ,  0.68631065,  0.68237865,
         0.67869711,  0.67525977,  0.67206055,  0.66909444,  0.66635644,
         0.66384226,  0.66154772,  0.65946907,  0.65760314,  0.65594703,
         0.65449816,  0.65325409,  0.65221

In [25]:
merged_obs, rewards, dones, infos = env.step({"test1": np.array([0.3, 2.0]), "test2": np.array([0.3, 2.0])})

({'test1': array([ 2.07239079,  1.9686743 ,  1.87538815,  1.7910645 ,  1.71449697,
          1.64468825,  1.58080471,  1.52214396,  1.46811247,  1.418203  ,
          1.37197936,  1.32906592,  1.28913593,  1.25190568,  1.21712589,
          1.18457735,  1.15406668,  1.12542272,  1.09849238,  1.07314026,
          1.04924405,  1.02669501,  1.0053947 ,  0.98525411,  0.96619314,
          0.94813895,  0.93102568,  0.91479284,  0.89938569,  0.88475376,
          0.87085146,  0.85763651,  0.84507006,  0.83311677,  0.82174385,
          0.81092083,  0.80061984,  0.79081529,  0.78148299,  0.77260107,
          0.76414883,  0.75610757,  0.7484591 ,  0.74118745,  0.73427731,
          0.72771448,  0.72148573,  0.71557868,  0.70998228,  0.70468569,
          0.69967902,  0.69495314,  0.6904996 ,  0.68631065,  0.68237865,
          0.67869711,  0.67525977,  0.67206055,  0.66909444,  0.66635644,
          0.66384226,  0.66154772,  0.65946907,  0.65760314,  0.65594703,
          0.65449816,  0.6532